In [11]:
import pandas as pd
import numpy as np

# Load Data

### South American Countries according to OWID

In [12]:
# According to Wikipedia, South Sandwich Islands are uninhabited, and South Georgia island has a very small,
# non-permanent population. Additionally, this country does not appear in the WHO data, so I will exclude it.
data = pd.read_csv("data/continents_owid.csv")
codes = data[data.Continent == "South America"].Code.values
countries = data[data.Continent == "South America"].Entity.values
countries = set(countries[countries != "South Georgia and the South Sandwich Islands"])

### WHO Population Data

In [13]:
# Keep only relevant columns
who_pop = pd.read_excel(
    io="data/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx",
    sheet_name="Estimates",
    skiprows=16,
    usecols=[
    "Region, subregion, country or area *",
    "Female Population, as of 1 July (thousands)",
    "Year",
    "Female Deaths (thousands)"]
)
# For ease of use
who_pop.rename(
    columns={
        "Region, subregion, country or area *": "Region",
        "Female Population, as of 1 July (thousands)": "FemPop1k",
        "Female Deaths (thousands)": "FemDeaths1k"},
    inplace=True
)

In [14]:
# only consider the years included in the Google Form
years = [1970, 1979, 1986, 2011, 2017]
who_pop = who_pop[who_pop["Year"].isin(years)]

Most South American countries (according to OWID) are also in the WHO data, except
"South Georgia and the South Sandwich Islands". A few countries have slightly different names, for instance:
 - "Venezuela" -> "Venezuela (Bolivarian Republic of)"
 - "Bolivia" -> "Bolivia (Plurinational State of)"
 - "Falkland Islands" -> "Falkland Islands (Malvinas)"

In [15]:
# Alternative names for a few South American countries
south_america = countries.union({
    'Venezuela (Bolivarian Republic of)',
    'Falkland Islands (Malvinas)',
    'Bolivia (Plurinational State of)'})
who_pop = who_pop[who_pop["Region"].isin(south_america)]  # filtered for south america

# Rename countries with alternative names (for simplicity)
who_pop.loc[who_pop["Region"] == "Venezuela (Bolivarian Republic of)", "Region"] = "Venezuela"
who_pop.loc[who_pop["Region"] == "Falkland Islands (Malvinas)", "Region"] = "Falkland Islands"
who_pop.loc[who_pop["Region"] == "Bolivia (Plurinational State of)", "Region"] = "Bolivia"

In [16]:
# Convert Year -> Int, and populations to float (we will divide later)
who_pop = who_pop.convert_dtypes(convert_string=False)

In [17]:
# Transform population/1000 and deaths/1000 into absolute values
who_pop["FemPop1k"] *= 1000
who_pop["FemDeaths1k"] *= 1000
# Rename them accordingly
who_pop.rename(columns={"FemPop1k": "TotFemPop", "FemDeaths1k": "TotFemDeaths"}, inplace=True)

### WHO Mortality Data

Notice that in the WHO mortality data we are missing three countries
1. Bolivia
2. Falkland Islands
3. South Georgia and the South Sandwich Islands (which has no population anyway)

I am not sure why Bolivia is missing. The last two, luckily, contribute very little to the overall South American population and data in general.

The important column is `Number` since it gives the absolute number of female deaths due to maternal conditions.

In [18]:
mortality = pd.read_csv(
    "data/WHOMortalityDatabase_Map_Maternal conditions_28th March 2024 17_52.csv", skiprows=6, index_col=False
)
# We only want to keep South American countries, for the years quote in the Google Form
# For all age groups and for female
mortality = mortality.query(
    "Year in @years & `Country Name` in @south_america & Sex == 'Female' & `Age Group` == '[All]'"
)
# Keep only relevant columns
mortality = mortality[
    ["Country Name", "Year", "Number", "Percentage of cause-specific deaths out of total deaths",
    "Death rate per 100 000 population"]]
# Remove index, useless in this case
mortality.reset_index(drop=True, inplace=True)
# Rename venezuela
mortality.loc[mortality["Country Name"] == "Venezuela (Bolivarian Republic of)", "Country Name"] = "Venezuela"

# Computation

I am assuming that the task is to compute the following metric
$$
\frac{\text{# maternal deaths in South America}}{\text{total female deaths in South America}} \times 100\,000
$$
for each year.

In [19]:
# Total South American population by year
pop_by_year = who_pop.groupby("Year")["TotFemPop"].sum()

In [20]:
def check(year):
    # countries that have mortality data 
    countries_with_data = mortality[mortality["Year"] == year]["Country Name"].values
    # Filter out WHO data to compute population with data
    df = who_pop[who_pop["Year"] == year]
    pop_with_data =  df[df["Region"].isin(countries_with_data)].TotFemPop.sum()
    return (pop_with_data / pop_by_year[year]) >= 0.8

In [21]:
action = ["Not Enough Data", "Enough Data"]
for ix, year in enumerate(years):
    print("Year ", year, ": ", action[int(check(year))])

Year  1970 :  Not Enough Data
Year  1979 :  Not Enough Data
Year  1986 :  Enough Data
Year  2011 :  Enough Data
Year  2017 :  Enough Data


Therefore I only need to compute it for the latest three years: 1986, 2011, 2017. It makes sense that data reporting got better over time.

In [22]:
years_with_80pc = years[2:]

#### 1986

In [23]:
# (Absolute) number of deaths due to maternal conditions by year for South America
n_deaths_mat_cond_by_year = mortality.query("Year in @years_with_80pc").groupby("Year")["Number"].sum()

In [576]:
tot_deaths_by_year = who_pop[who_pop["Year"].isin(years_with_80pc)].groupby("Year")["TotFemDeaths"].sum()

In [582]:
crude_dr_100k = ((n_deaths_mat_cond_by_year / tot_deaths_by_year) * 100000).round(2)

In [583]:
crude_dr_100k

Year
1986    395.89
2011     313.4
2017    273.09
dtype: Float64